# 1. 填充
- 无填充时：
    - 第1层输出大小 = 32 - 5 + 1 = 28
    - 第2层输出大小 = 28 - 5 + 1 = 32 - 2 x (5-1) = 24
    - 第3层输出大小 = 24 - 5 + 1 = 32 - 3 x (5-1) = 20
    - 第4层输出大小 = 20 - 5 + 1 = 32 - 4 x (5-1) = 16
    - 第5层输出大小 = 16 - 5 + 1 = 32 - 5 x (5-1) = 12
    - 第6层输出大小 = 12 - 5 + 1 = 32 - 6 x (5-1) = 8
    - 第7层输出大小 = 8 - 5 + 1 = 32 - 7 x (5-1) = 4
- 为了避免网络越深，输出大小过小，解决办法：填充
<img src="./pic/填充.PNG" width=300 height=300>

## 在输入周围添加额外的行/列
- <img src="./pic/在输入周围添加0.PNG" width=400 height=400>
- 任意的边界填充，无跨步（Arbitrary padding, no strides）
<img src="./pic/任意的边界填充，无跨步.gif" width=200 height=200>
- 半填充，无跨步（Half padding, no strides）
<img src="./pic/半填充，无跨步.gif" width=200 height=200>
- 全填充，无跨步（Full padding, no strides）
<img src="./pic/全填充，无跨步.gif" width=200 height=200>

## 填充shape
- p为padding
- 通过填充，输出和输入的形状不会发生变化
- k<sub>h</sub>很少为偶数，当为偶数时，上侧多填充一行，下侧少填充一行；反之也行。
<img src="./pic/填充shape.PNG" width=300 height=300>
- 因为padding=kernel-1，而padding是分在图片上下的，kernel为基数，padding就可以对半分。

# 2. 步幅

- 224 - 层数(5-1) = 4, 层数 = 55. 
<img src="./pic/步幅1.PNG" width=400 height=400>
- 之前输出大小与层数线性相关，步幅可以变成指数相关

#### 例子
- padding 为 2
- 横向：宽度2，往右走2行
- 纵向：高度3，往下走3行
<img src="./pic/步幅例子.PNG" width=400 height=400>

- 没有边界填充，跨步（No padding, strides）
<img src="./pic/没有边界填充，跨步.gif" width=200 height=200>
- 有边界填充，跨步（Padding, strides）
<img src="./pic/有边界填充，跨步.gif" width=200 height=200>
- 有填充，跨步（输入图像变长为偶数）（Padding, strides (odd)）
<img src="./pic/有填充，跨步.gif" width=200 height=200>

## 步幅输出形状
<img src="./pic/步幅输出形状.PNG" width=400 height=400>


- Q：为什么通常用3x3的卷积核？3个像素的视野很小？
- A： 第一层如果是3x3的filter的话，第二层中一块3x3的区域就包含了输入中一块5x5的区域的信息(stride=1)。假如最后一层的输出为1x1的区域，那么该区域包含整个输入信息

- Q：有声明办法能让超参数也一起训练
- A：Neural Network Architecture Search(NAS)

- Q：自动训练参数，是否会更容易过拟合？
- A：验证集设置比较好的话，可以很好的避免过拟合。验证集不好的话，容易过拟合

- 3x3卷积核，10层神经网络，效果等价于5x5卷积核，5-6层神经网络。但是3x3计算更快

# 总结

- 填充和步幅时卷积层的超参数（包括卷积核的大小）
- 填充在输入周围添加额外的行/列，来控制输出形状的减少量
    - 让输出保持不变，或者变大
    - 当输入图片比较小时，避免卷积层减少输出，以便传入比较深的模型
- 步幅时每次滑动核窗口时的行/列的步长，可以成倍减少输出形状
    - 当输入图像比较大时，通过步幅能减少计算量

# 填充核步幅的代码实现

- 在所有侧边填充1个像素

In [2]:
import torch
from torch import nn


# 为了方便起见，我们定义了一个计算卷积层的函数。
# 此函数初始化卷积层权重，并对输入和输出提高和缩减相应的维数
def comp_conv2d(conv2d, X):
    # 这里的（1，1）表示批量大小和通道数都是1
    X = X.reshape((1, 1) + X.shape)    # 【在维度前面加入一个批量大小数核通道数】
    Y = conv2d(X)
    # 省略前两个维度：批量大小和通道
    return Y.reshape(Y.shape[2:])

# 请注意，这里每边都填充了1行和1列，因此总共添加了2行和2列
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1)
X = torch.rand(size=(8, 8))
comp_conv2d(conv2d, X).shape        # 8 - 5 + 【2】 + 1 = 8

torch.Size([8, 8])

- 填充不同的高度和宽度
    - 使输出形状等于输入形状

In [3]:
conv2d = nn.Conv2d(1, 1, kernel_size=(5, 3), padding=(2, 1))
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

- 将高度和宽度的步幅设置为2，从而将输入的高度和宽度减半
    - 第一个1为输入通道，第二个1为输出通道
    - 笔记21：多输入通道与多输出通道

In [4]:
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1, stride=2)
comp_conv2d(conv2d, X).shape    # （8-3+2+2）/ 2 = 4.5，取整4

torch.Size([4, 4])

- 一个稍微复杂的例子

In [5]:
conv2d = nn.Conv2d(1, 1, kernel_size=(3, 5), padding=(0, 1), stride=(3, 4))
comp_conv2d(conv2d, X).shape    # w: (8 - 3 + 0 + 3) / 3 = 2（取整）
                                # h：(8 - 5 + 2 + 4) / 4 = 2

torch.Size([2, 2])